# Métricas
---
Las metricas utilizadas en *Forescast data driven models* son diferentes a las utilizadas en otras áreas de *Deep learning*.

## latitude-weighted Root Mean Square Error (RMSE)
---
Se calculas para cada varaible para un nivel verical dado (i.e. $T500$ temperatura a 500 m de altura $RMSE_{500}$).

Es el RMSE pero promediado a traves de las dimensiones:
> * Longitud $j$: $[1, \ldots, J]$ 
> * Latitud: $i$: $[1, \ldots, I]$
> * Tiempo: $t$: $[1, \ldots, T]$

$$

\mathrm{RMSE}_l=\sqrt{\frac{1}{T I J} \sum_t^T \sum_i^I \sum_j^J w(i)\left(f_{t, i, j}-o_{t, i, j}\right)^2}

$$

> **Donde:**
> * $f$: Forecast 
> * $o$: Ground truth (ERA 5) 
> * $w$: Latitud weights 

**Latitud weights:** Debido a que las celdas en altas latitudes son mas pequeñas comparadas con las de bajas latitudes se pondera el error de la siguiente forma:

$$

w(i)=\frac{\sin \theta_i^{\mathrm{u}}-\sin \theta_i^{\mathrm{l}}}{\frac{1}{I} \sum_i^I\left(\sin \theta_i^{\mathrm{u}}-\sin \theta_i^{\mathrm{l}}\right)}

$$

Donde $θ^u_i$ and $θ^l_i$ indican los límites superior e inferior de la celda con indice de latitud $i$ respectivamente.

In [ ]:
# !pip install git+https://github.com/google-research/weatherbench2.git

In [9]:
import xarray as xr
from weatherbench2 import metrics

In [3]:
data_dir = "C:\\Users\\gcuervo\\OneDrive - Universidad de Las Palmas de Gran Canaria\\Documents\\Doctorado\\DB\\GraphCast_data\\"

In [13]:
ground_truth = (xr.load_dataset(data_dir + "eval_targets.nc")
                .isel(time=slice(0,6))
                .rename({'lon': 'longitude','lat': 'latitude'})
                )
ground_truth

<xarray.Dataset>
Dimensions:                  (batch: 1, time: 6, latitude: 181, longitude: 360,
                              level: 13)
Coordinates:
  * longitude                (longitude) float32 0.0 1.0 2.0 ... 358.0 359.0
  * latitude                 (latitude) float32 -90.0 -89.0 -88.0 ... 89.0 90.0
  * time                     (time) timedelta64[ns] 06:00:00 ... 1 days 12:00:00
  * level                    (level) int32 50 100 150 200 ... 700 850 925 1000
Dimensions without coordinates: batch
Data variables:
    2m_temperature           (batch, time, latitude, longitude) float32 248.3...
    mean_sea_level_pressure  (batch, time, latitude, longitude) float32 9.983...
    10m_v_component_of_wind  (batch, time, latitude, longitude) float32 -0.04...
    10m_u_component_of_wind  (batch, time, latitude, longitude) float32 0.048...
    total_precipitation_6hr  (batch, time, latitude, longitude) float32 7.443...
    temperature              (batch, time, level, latitude, longitude) float32 ...
    geopotential             (batch, time, level, latitude, longitude) float32 ...
    u_component_of_wind      (batch, time, level, latitude, longitude) float32 ...
    v_component_of_wind      (batch, time, level, latitude, longitude) float32 ...
    vertical_velocity        (batch, time, level, latitude, longitude) float32 ...
    specific_humidity        (batch, time, level, latitude, longitude) float32 ...

In [14]:
forecast = xr.load_dataset(data_dir + "predictions.nc").rename({'lon': 'longitude','lat': 'latitude'})
forecast

<xarray.Dataset>
Dimensions:                  (time: 6, batch: 1, latitude: 181, longitude: 360,
                              level: 13)
Coordinates:
  * time                     (time) timedelta64[ns] 06:00:00 ... 1 days 12:00:00
  * longitude                (longitude) float32 0.0 1.0 2.0 ... 358.0 359.0
  * latitude                 (latitude) float32 -90.0 -89.0 -88.0 ... 89.0 90.0
  * level                    (level) int32 50 100 150 200 ... 700 850 925 1000
Dimensions without coordinates: batch
Data variables:
    10m_u_component_of_wind  (time, batch, latitude, longitude) float32 -0.37...
    10m_v_component_of_wind  (time, batch, latitude, longitude) float32 -0.28...
    2m_temperature           (time, batch, latitude, longitude) float32 247.8...
    geopotential             (time, batch, level, latitude, longitude) float32 ...
    mean_sea_level_pressure  (time, batch, latitude, longitude) float32 9.95e...
    specific_humidity        (time, batch, level, latitude, longitude) float32 ...
    temperature              (time, batch, level, latitude, longitude) float32 ...
    total_precipitation_6hr  (time, batch, latitude, longitude) float32 6.125...
    u_component_of_wind      (time, batch, level, latitude, longitude) float32 ...
    v_component_of_wind      (time, batch, level, latitude, longitude) float32 ...
    vertical_velocity        (time, batch, level, latitude, longitude) float32 ...

In [18]:
RMSE_l = (metrics.MSE().compute(forecast, ground_truth))**0.5
RMSE_l

<xarray.Dataset>
Dimensions:                  (level: 13, batch: 1)
Coordinates:
  * level                    (level) int32 50 100 150 200 ... 700 850 925 1000
Dimensions without coordinates: batch
Data variables:
    10m_u_component_of_wind  (batch) float32 4.279
    10m_v_component_of_wind  (batch) float32 4.882
    2m_temperature           (batch) float32 4.227
    geopotential             (batch, level) float32 491.4 602.6 ... 531.4 556.1
    mean_sea_level_pressure  (batch) float32 713.2
    specific_humidity        (batch, level) float32 6.378e-08 ... 0.001644
    temperature              (batch, level) float32 1.916 2.407 ... 3.856 3.685
    total_precipitation_6hr  (batch) float32 0.002875
    u_component_of_wind      (batch, level) float32 4.056 6.724 ... 6.144 4.712
    v_component_of_wind      (batch, level) float32 4.411 7.021 ... 7.035 5.482
    vertical_velocity        (batch, level) float32 0.0131 0.03589 ... 0.1173

<xarray.Dataset>
Dimensions:                  (level: 13, batch: 1)
Coordinates:
  * level                    (level) int32 50 100 150 200 ... 700 850 925 1000
Dimensions without coordinates: batch
Data variables:
    10m_u_component_of_wind  (batch) float32 4.279
    10m_v_component_of_wind  (batch) float32 4.882
    2m_temperature           (batch) float32 4.227
    geopotential             (batch, level) float32 491.4 602.6 ... 531.4 556.1
    mean_sea_level_pressure  (batch) float32 713.2
    specific_humidity        (batch, level) float32 6.378e-08 ... 0.001644
    temperature              (batch, level) float32 1.916 2.407 ... 3.856 3.685
    total_precipitation_6hr  (batch) float32 0.002875
    u_component_of_wind      (batch, level) float32 4.056 6.724 ... 6.144 4.712
    v_component_of_wind      (batch, level) float32 4.411 7.021 ... 7.035 5.482
    vertical_velocity        (batch, level) float32 0.0131 0.03589 ... 0.1173